In [46]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.llms.huggingface_hub import HuggingFaceHub

In [2]:
summary_path = r"data\summaries"

loader = PyPDFDirectoryLoader(path=summary_path)
docs = loader.load()

In [26]:
print(type(docs), len(docs), type(docs[0]))

<class 'list'> 16 <class 'langchain_core.documents.base.Document'>


In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=400,
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_documents(docs)

In [28]:
print(type(chunks), len(chunks), type(chunks[0]))

<class 'list'> 59 <class 'langchain_core.documents.base.Document'>


In [34]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", encode_kwargs = {'normalize_embeddings': True})

faiss = FAISS.from_documents(documents=chunks, embedding=embedding_model)

In [36]:
relevant_context = faiss.similarity_search_with_relevance_scores(query="Who was the wife of king Santanu?", k=3)

In [43]:
for context in relevant_context:
    print(f"Similarity score: {context[1]}\t Source: {context[0].metadata}")
    print(context[0].page_content)
    print("-"*100)

Similarity score: 0.35486126656368644	 Source: {'source': 'data\\summaries\\mahabharat_summary.pdf', 'page': 0, 'start_index': 1598}
but only if Satyavathi’s children inherit, leaving poor Bhishma out in the cold.  Bhishma was actually cool with this and said he would remain celibate so that he never had children.  Thus, King Santanu and ferry woman Satyavathi married.  They had two boys: one had no children and died in battle, and one (Vichitravirya) grew to adulthood and married two women (Ambika and Ambalika).  But before either of his wives had children, Vichitravirya died and not long after that King Santanu also died.  Thus, the only surviving member of the royal family was Bhishma who had taken a vow of celibacy and refused to break it.  What Queen Satyavathis had not told anyone that before she was married she had actually been born from a fish and had had an encounter with a sage and given birth to a son named Vyasa.  So even though Vyasa isn’t exactly the heir, he can still k

In [44]:
HUGGINGFACEHUB_API_TOKEN = getpass()

 ········


In [45]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [47]:
def get_conversation_chain(vector_store):
    repo_id="google/gemma-1.1-2b-it"
    llm = HuggingFaceHub(repo_id=repo_id, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,  model_kwargs={'temperature':0.5, 'max_length':128})
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    retriever = vector_store.as_retriever()
    conv_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
    )
    return conv_chain

In [49]:
conversation_chain = get_conversation_chain(faiss)

C:\Users\rachi\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [50]:
query = "Who was king Santanu's second wife and who were their childrens?"
response = conversation_chain({'question':query})

C:\Users\rachi\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [51]:
response['chat_history'][1].pretty_print()

================================== Ai Message ==================================

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

but only if Satyavathi’s children inherit, leaving poor Bhishma out in the cold.  Bhishma was actually cool with this and said he would remain celibate so that he never had children.  Thus, King Santanu and ferry woman Satyavathi married.  They had two boys: one had no children and died in battle, and one (Vichitravirya) grew to adulthood and married two women (Ambika and Ambalika).  But before either of his wives had children, Vichitravirya died and not long after that King Santanu also died.  Thus, the only surviving member of the royal family was Bhishma who had taken a vow of celibacy and refused to break it.  What Queen Satyavathis had not told anyone that before she was married she had actually been born from a fish and had had an encount

In [52]:
query = "So what happened to the second child of santanu and satyavathi (Ambalika)?"
response = conversation_chain({'question':query})

In [53]:
response['chat_history'][3].pretty_print()

================================== Ai Message ==================================

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

but only if Satyavathi’s children inherit, leaving poor Bhishma out in the cold.  Bhishma was actually cool with this and said he would remain celibate so that he never had children.  Thus, King Santanu and ferry woman Satyavathi married.  They had two boys: one had no children and died in battle, and one (Vichitravirya) grew to adulthood and married two women (Ambika and Ambalika).  But before either of his wives had children, Vichitravirya died and not long after that King Santanu also died.  Thus, the only surviving member of the royal family was Bhishma who had taken a vow of celibacy and refused to break it.  What Queen Satyavathis had not told anyone that before she was married she had actually been born from a fish and had had an encount